## 2H 2015 Full Market by Market Analysis All Parameters

The following cell runs through all RootMetric Sprint data in the Sprint_Detail.csv files.  It will sift through all subdirectories and first look for *.zip files then look *.csv files with a specific name.  All the statistics are calculated through every run. Keep in mind there are modifications here to process the differences in the 2H 2015 files from the 1H 2015 files.

In [4]:
import os
import get_stats
import get_ss_stats
import get_root_dataframe
import calc_distance as cd
import pandas as pd
import time

%matplotlib inline
# Valid values for type_flag = 'FAILED', 'ALL', 'CALL'

stat_list =[]
#'city':;'mean':''max':''min':
pt_ct = 0

for path, subdirs, files in os.walk("."):
        for name in files:
            if name.endswith(".zip"):
                f = os.path.join(path,name)
                # Careful @ this point the path is specific to the directory which will change
                # depending on what system the data files are located.
                head,filedate = os.path.split(path)
                if True:
                    _stat= {}
                    pt_ct = 0
                    # Get the basic file and base dataframe:
                    df,filelist = get_root_dataframe.get_rm_df(f)                      
                    # Get the CDMA status for failed calls:
                    
                    pt_ct +=1
                    stat_name = 'CDMA_Signal_Strength'                    
                    nfi, failed_stats, dm_call, call_duration = get_stats.get_ss_stats(df.copy(), stat_name)
                    print '\n', pt_ct, 'CDMA Failure Signal Strengths Mean, Max, Min std', failed_stats

                    pt_ct +=1
                    nfi, all_stat_CDMA = get_stats.get_gen_stat(df.copy(),stat_name)
                    print pt_ct,'%s all Mean, Max, Min, Std'%stat_name, all_stat_CDMA
                    
                    # Get the failed RSRP stats:
                    type_flag ='FAILED'
                    stat_name = 'LTE_RSRP'

                    pt_ct +=1
                    nfi, all_stat_RSRP = get_stats.get_gen_stat(df.copy(),stat_name)
                    print pt_ct,'%s all Mean, Max, Min, Std'%stat_name, all_stat_RSRP
                  
                    stat_name = 'LTE_RSRP'
                    pt_ct +=1
                    nfi, failed_stats2, dm_fail = get_stats.get_lte_fail_stats(df.copy(), type_flag,stat_name)
                    print pt_ct,'RSRP Failure Signal Strengths Mean, Max, Min Std', failed_stats2
                    
                    stat_name = 'LTE_RSRQ'
                    pt_ct +=1
                    nfi, all_stat_RSRQ = get_stats.get_gen_stat(df.copy(),stat_name)
                    print pt_ct,stat_name, all_stat_RSRQ
                    
                    pt_ct +=1
                    nfi, fail_stat_RSRQ, dm_fail2 = get_stats.get_lte_fail_stats(df.copy(), type_flag, stat_name)
                    print pt_ct,stat_name, type_flag, fail_stat_RSRQ
                    
                    stat_name = 'LTE_RSSNR'
                    pt_ct +=1
                    nfi, all_stat_RSSNR = get_stats.get_gen_stat(df.copy(),stat_name)
                    print pt_ct,stat_name, all_stat_RSSNR     

                    # Let's get any stat No filter for failures
                    #LTE_RSRP	LTE_RSRQ	LTE_RSSNR	LTE_CQI	LTE_RSSI
                    #CDMA_Signal_Strength	CDMA_Ecio             

                    try:
                        # Calculate the throughput on technology for downlink

                        my_type, tp_speed = get_stats.get_tput_count(df,'Downlink_Throughput:Network_Types', \
                                                                     'Downlink_Throughput:Final_Test_Speed')
                        _stat['RMmetro'] = filelist
                        _stat['tolte'] = my_type[0]
                        _stat['toehrpd'] = my_type[1]
                        _stat['tomix'] = my_type[2]
                        _stat['toreva'] = my_type[3]
                        _stat['tortt'] = my_type[4]
                        _stat['tototal'] =my_type[5]
                        
                        _stat['tplte_mean'] = tp_speed[0]
                        _stat['tpehrpd_mean'] = tp_speed[2]
                        _stat['tpmix_mean'] = tp_speed[4]
                        
                        _stat['tplte_med'] = tp_speed[1]
                        _stat['tpehrpd_med'] = tp_speed[3]
                        _stat['tpmix_med'] = tp_speed[5]
                        
                        _stat['tp_median'] = tp_speed[6]
                        _stat['tp_max'] = tp_speed[10]
                        _stat['tp_acc_dur'] = tp_speed[11]
                        _stat['tp_ttfb'] = tp_speed[12]
 
                        pt_ct +=1
                        print pt_ct,'DL Network Type Counts LTE, eHRPD, Mix, RevA, RTT, Total', my_type
                        pt_ct +=1
                        print pt_ct,'DL Network Type Speeds LTE (mean,median), eHRPD, Mix, Total median,Tput Max, \
                        Access Duration, Time for First Byte',tp_speed
                        
                        # Calculate the throughput on technology for uplink
                        my_type, tp_speed = get_stats.get_tput_count(df,'Uplink_Throughput:Network_Types', \
                                                                     'Uplink_Throughput:Final_Test_Speed')
                        _stat['RMmetro'] = filelist
                        _stat['utolte'] = my_type[0]
                        _stat['utoehrpd'] = my_type[1]
                        _stat['utomix'] = my_type[2]
                        _stat['utoreva'] = my_type[3]
                        _stat['utortt'] = my_type[4]
                        _stat['utototal'] =my_type[5]
                        
                        _stat['utplte_mean'] = tp_speed[0]
                        _stat['utpehrpd_mean'] = tp_speed[2]
                        _stat['utpmix_mean'] = tp_speed[4]
                        
                        _stat['utplte_med'] = tp_speed[1]
                        _stat['utpehrpd_med'] = tp_speed[3]
                        _stat['utpmix_med'] = tp_speed[5]
                        
                        _stat['utp_median'] = tp_speed[6]
                        _stat['ultp_max'] = tp_speed[10]
                        _stat['ultp_acc_dur'] = tp_speed[11]
                        _stat['ultp_ttc'] = tp_speed[12]
 
                        pt_ct +=1
                        print pt_ct,'UL Network Type Counts LTE, eHRPD, Mix, RevA, RTT, Total', my_type
                        pt_ct +=1
                        print pt_ct,'UL Network Type Speeds LTE (mean,median), eHRPD, Mix, Total median,Tput Max,Access Duration, Time for First Byte', tp_speed
                        
                    except:
                        print 'Can not get throughput metrics'

                    try:
                        # Check for unique eNBID count
                        
                        dummy, enbid_count = get_stats.get_enbid_count(df)
                        
                        _stat['enbid_count'] = enbid_count
                        pt_ct +=1
                        print pt_ct,'Unique LTE eNB Ids ', enbid_count
                        
                        # Call function to count unique eNB ids
                        enb_dict, enbid_count = get_stats.get_enbid_count(df)
                        # Call function to get bounding box of drive test + area
                        box, area = cd.get_box(df)
                        pt_ct +=1
                        print pt_ct,'eNB per SqM ', enbid_count/area                        
                        _stat['mkarea'] = area
                    except:
                        print 'Can not get eNB counts'
                        
                        
                    # Convert objects to numeric values where it makes sense.  
                    # Careful, this might disturb some assumptions about the data
                    # Also, in the first implementation we calculated distance from base stations of 
                    # only failed calls.  We want to look at this for all calls.
                    try:
                        dm_call = dm_call.convert_objects(convert_numeric=True)
                        # Try all calls!
                        #dm_call = df.convert_objects(convert_numeric=True)
                        dm_call.dropna(subset=['Base_Station_ID'], how ='any', inplace =1)
 
                        clean_bs_dict = cd.create_base_station_dict(dm_call)
                        cdma_avg_dist = cd.get_average_distance(clean_bs_dict,dm_call)
 
 
                        pt_ct +=1
                        print pt_ct, 'Average Distance of Failed CDMA calls to cells = %8.2f Km'%cdma_avg_dist

                        # Convert objects to numeric values where it makes sense.  
                        # Careful, this might disturb some assumptions about the data
                        # Turns out there are no coordinates saved for LTE base stations.  So this data is meaningless.
                        dm_fail.dropna(subset=['Base_Station_ID'], how ='any', inplace =1)
                        dm_fail = dm_fail.convert_objects(convert_numeric=True)
                        clean_bs_dict = cd.create_base_station_dict(dm_fail)
                        lte_avg_dist = cd.get_average_distance(clean_bs_dict,dm_fail)
                        pt_ct +=1
                        print pt_ct,'Average Distance of', type_flag,' LTE Data to cells = %8.2f Km\n'%lte_avg_dist
                        
                        # Save calculated distances.
                        _stat['RMmetro'] = filelist
                        _stat['lte_distance'] = lte_avg_dist
                        _stat['cdma_distance'] = cdma_avg_dist
                    except:
                        print 'distance metrics failed'
                    

                    if  failed_stats:
    
                        _stat['RMmetro'] = filelist
                        _stat['CDMA_Fail_mean'] = failed_stats[0]
                        _stat['CDMA_Fail_max'] = failed_stats[1]
                        _stat['CDMA_Fail_min'] = failed_stats[2]
                        _stat['CDMA_Fail_std'] = failed_stats[3]
                        
                    if   failed_stats2:
                        _stat['RMmetro'] = filelist
                        _stat['RSRP_Fail_mean'] = failed_stats2[0]
                        _stat['RSRP_Fail_max'] = failed_stats2[1]
                        _stat['RSRP_Fail_min'] = failed_stats2[2]
                        _stat['RSRP_Fail_std'] = failed_stats2[3]
 
 
                        _stat['RSRP_mean'] = all_stat_RSRP[0]
                        _stat['RSRP_max'] = all_stat_RSRP[1]
                        _stat['RSRP_min'] = all_stat_RSRP[2]
                        _stat['RSRP_std'] = all_stat_RSRP[3]

                        _stat['RSRQ_mean'] = all_stat_RSRQ[0]
                        _stat['RSRQ_max'] = all_stat_RSRQ[1]
                        _stat['RSRQ_min'] = all_stat_RSRQ[2]
                        _stat['RSRQ_std'] = all_stat_RSRQ[3]

                        _stat['RSSNR_mean'] = all_stat_RSSNR[0]
                        _stat['RSSNR_max'] = all_stat_RSSNR[1]
                        _stat['RSSNR_min'] = all_stat_RSSNR[2]
                        _stat['RSSNR_std'] = all_stat_RSSNR[3]

                        _stat['CDMA_mean'] = all_stat_CDMA[0]
                        _stat['CDMA_max'] = all_stat_CDMA[1]
                        _stat['CDMA_min'] = all_stat_CDMA[2]
                        


                    stat_list.append(_stat)

########################

print '\n\nWe are done Processing'

Charleston-SC_2015-2H_Sprint_Detail.csv

1 CDMA Failure Signal Strengths Mean, Max, Min std [-80.378787878787875, -59.0, -120.0, 15.217379569262397]
2 CDMA_Signal_Strength all Mean, Max, Min, Std [-101.50467487563067, -39.0, -120.0, 22.276525583874516, -120.0]
3

get_stats.py:54: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dm_fail.dropna(subset=['CDMA_Signal_Strength'], how ='any', inplace =1)
get_stats.py:69: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dm_sig.dropna(subset=['Outgoing_Call:Call_Setup_Duration'], how ='any', inplace =1)
get_stats.py:152: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dm_fail.dropna(subset=[stat_name], how ='any', inplace =1)


 LTE_RSRP all Mean, Max, Min, Std [-96.392006535536979, -57.0, -124.0, 10.375500897163812, -98.0]
4 RSRP Failure Signal Strengths Mean, Max, Min Std [-96.517610062893084, -68.0, -121.0, 10.267216157339533, -98.0]
5 LTE_RSRQ [-9.8390808398082328, -3.0, -15.0, 2.5123652307433848, -10.0]
6 LTE_RSRQ FAILED [-10.958145524790728, -3.0, -15.0, 2.2241256715049604, -11.0]
7

get_stats.py:271: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  lte_type.dropna(subset=[stat_name1], how ='any', inplace =1)
get_stats.py:275: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  eHRPD_type.dropna(subset=[stat_name1], how ='any', inplace =1)
get_stats.py:280: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  mix_type.dropna(subset=[stat_name1], how ='any', inplace =1)


 LTE_RSSNR [10.590157962503534, 30.0, -18.399999999999999, 9.4934919792607353, 10.4]
8 DL Network Type Counts LTE, eHRPD, Mix, RevA, RTT, Total (531, 59, 4, 0, 10, 590)
9 DL Network Type Speeds LTE (mean,median), eHRPD, Mix, Total median,Tput Max,                         Access Duration, Time for First Byte (6958.8138957771498, 5337.5095614776101, 467.78520234235276, 296.801859305081, 1723.4761228422633, 132.20249912839449, 4584.633135328525, 302.43915409186479, 9302.5871551290184, 15941.105822409343, 27856.355828534899, 0.2495, 0.41425000000000001)
10 UL Network Type Counts LTE, eHRPD, Mix, RevA, RTT, Total (536, 64, 1, 0, 7, 595)
11 UL Network Type Speeds LTE (mean,median), eHRPD, Mix, Total median,Tput Max,Access Duration, Time for First Byte (3872.6107166521169, 3529.05738413579, 834.04948123176564, 972.696367070326, 441.18742361959801, 441.18742361959801, 3090.7712476391498, 714.83186817917931, 5425.8404977639402, 6829.9120171848535, 7762.3751111111096, 0.058000000000000003, 0.057

c:\Anaconda\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


## This Cell will write out the complete set of stats to a csv file.

In [2]:
# This Cell is used for Printing.  Eventually we will just create a subfuction print _stat
 
import csv
csv_file ='../drive-report/dontuse_stats-2h.csv'
csv_columns = ['RMmetro','RSRP_mean','RSRP_max','RSRP_min','RSRP_std',\
               'RSRQ_mean','RSRQ_max','RSRQ_min','RSRQ_std',\
               'RSSNR_mean','RSSNR_max','RSSNR_min', 'RSSNR_std', \
               'CDMA_mean', 'CDMA_max', 'CDMA_min', 'CDMA_std', \
               'RSRP_Fail_mean', 'RSRP_Fail_max', 'RSRP_Fail_min', 'RSRP_Fail_std',\
               'CDMA_Fail_mean', 'CDMA_Fail_max', 'CDMA_Fail_min','CDMA_Fail_std', \
               'tolte', 'toehrpd', 'tomix', 'toreva', 'tortt','tototal', \
               'tplte_mean', 'tpehrpd_mean', 'tpmix_mean', \
               'tplte_med','tpehrpd_med','tpmix_med', \
               'utolte', 'utoehrpd', 'utomix', 'utoreva', 'utortt','utototal', \
               'utplte_mean', 'utpehrpd_mean', 'utpmix_mean', \
               'utplte_med','utpehrpd_med','utpmix_med', \
               'lte_distance','cdma_distance',\
               'tp_median', 'utp_median', 'tp_max','ultp_max', 
                'tp_acc_dur', 'tp_ttfb','ultp_acc_dur','ultp_ttc','enbid_count','mkarea']

try:
    with open(csv_file, 'wb') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns,dialect='excel')
        writer.writeheader()
        for data in stat_list:
           writer.writerow(data)
except IOError as (errno, strerror):
        print("I/O error({0}): {1}".format(errno, strerror)) 	
       

# Market Listing of files in Directory

In [2]:
# code Fragment to list markets in director
import os
import get_stats
import get_ss_stats
import get_root_dataframe
import calc_distance as cd
import pandas as pd

%matplotlib inline
# Valid values for type_flag = 'FAILED', 'ALL', 'CALL'

stat_list =[]
#'city':;'mean':''max':''min':
pt_ct = 0

for path, subdirs, files in os.walk("."):
        for name in files:
            if name.endswith(".zip"):
                f = os.path.join(path,name)
                # Careful @ this point the path is specific to the directory which will change
                # depending on what system the data files are located.
                head,filedate = os.path.split(path)
                if True:
                    _stat= {}
                    pt_ct = 0
                    # Get the basic file and base dataframe:
                    df,filelist = get_root_dataframe.get_rm_df(f)                      
                    # Get the CDMA status for failed calls:

Charleston-SC_2015-2H_Sprint_Detail.csv
LasVegas-NV_2015-2H_Sprint_Detail.csv
Omaha-NE_2015-2H_Sprint_Detail.csv


In [3]:
# Read csv mapping file into Pandas
import os
import pandas as pd
import csv
filename = 'RM_MSA_LookUP.csv'
try:
    with open(filename) as mapfile:
        mapfile_df = pd.read_csv(mapfile,skipinitialspace=True,dtype=unicode)
except:
    print 'i/o error'